In [5]:
import qiskit
from qiskit import *
from qiskit import IBMQ
from qiskit.providers.aer.noise import NoiseModel

from matplotlib import pyplot as plt

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import state_fidelity
from qiskit import BasicAer
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor

# Import measurement calibration functions
from qiskit.ignis.mitigation.measurement import (complete_meas_cal, tensored_meas_cal,
                                                 CompleteMeasFitter, TensoredMeasFitter)

from qiskit import IBMQ, execute
from qiskit import QuantumCircuit

from qiskit.providers.aer import QasmSimulator
from qiskit.tools.visualization import plot_histogram

from qiskit.test.mock import FakeMontreal

import numpy as np
import scipy as sp
from scipy.special import expit, softmax #logistic fns 
from sklearn.metrics import confusion_matrix

import scipy.optimize as optimize
from sklearn import linear_model

import time
import itertools
from matplotlib import pyplot as plt
import matplotlib.lines as mlines

from qiskit.providers.ibmq.managed import IBMQJobManager
from qiskit.circuit.random import random_circuit


IBMQ.save_account('488a01d1ced1b582bb3182aa638da3e3571b07af5f757f78ad1d05a9f0dfec77bbbf44eb3563259f5238ff2b0fadd3963202ab72b45a65671caa9c47b492882f')
IBMQ.load_account()


import pickle
import cv2 


import time


configrc.store_credentials:WARNING:2021-04-24 20:48:51,807: Credentials already present. Set overwrite=True to overwrite.
ibmqfactory.load_account:WARNING:2021-04-24 20:48:52,044: Credentials are already in use. The existing account in the session will be replaced.


In [6]:
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)    

In [7]:
# Function that converts a 1D vectorized image into a (nr x nc) 2D array
def unpackcw(x,nr,nc):
	A = x.reshape(nc,nr)
	return A.T
	
# resize img 2D array to specified nr,nc
def resize_img(img,nr,nc):
	return cv2.resize(img, dsize=(nr, nc), interpolation=cv2.INTER_CUBIC)

def resize_dataset(data, o_size, nrc):
	
	new_data = np.zeros((nrc*nrc, len(data[0])))
	for i in range(len(data[0])):
		img = unpackcw(data[:,i],o_size,o_size)
		img = resize_img(img,nrc,nrc)
		img = img.T
		img = img.reshape(nrc*nrc)
		new_data[:,i] = img
	return new_data


def reshape_img_MNIST(data,size,nr,nc):
    
    reshaped_data = np.zeros((nr,nc))
    
    div = int(size/nr)
    for j in range(div):
        for i in range(size):  
            if (j%2 == 1):
                p = size-i-1
            else:
                p = i
                
            reshaped_data[:,j*size+i] = data[j*nr:(j+1)*nr,p]
            
    return reshaped_data

In [19]:
def circuit_build(data):
        
    nr = data.shape[0]
    nc = data.shape[1]

    nq = nr+1
    nmeas = int(nc/2)
        
    qr = QuantumRegister(nq)
    cr = ClassicalRegister(nmeas)

    circuit = QuantumCircuit(qr, cr) 

    i=0
    m=0
    while (i < nc):
    
        v1 = data[:,i]
        v2 = data[:,i+1]
         
        v = v1
        
        # H's
        for j in range(nq):
            circuit.h(qr[j])


        # U
        

        # ZZ's, pairs containing a measurement qubit have phi = pi/2
        for j in range(nq):
            for k in range(j,nq):
                phi = (np.pi/2)
                if (j != k):
                    if (j < nr and k < nr):
                        if (j < k):
                            phi = v[j]*(np.pi-v[k])
                        else:
                            phi = (np.pi-v[j]+v[k])
                    elif (j < nr and k >= nr): #try something with meas qubits
                        phi = v[j]*(np.pi-v[k-nr]+v[j])

                    # AVOID SWAPS
                    if ((j==0 and k==1) or (j==0 and k==14) or (j==1 and k==2) or (j==1 and k==13) or (j==2 and k==3) or
                        (j==2 and k==12) or (j==3 and k==4) or (j==3 and k==11) or (j==4 and k==5) or (j==4 and k==10) or
                        (j==5 and k==6) or (j==5 and k==9) or (j==6 and k==8) or (j==7 and k==8)):
                        
                        # ZZ gate
                        if (k != nq-1):
                            circuit.cx(qr[j], qr[k])
                            circuit.rz(2*phi, qr[k])
                            circuit.cx(qr[j], qr[k])
                        else:                           
                            circuit.cx(qr[k], qr[j])
                            circuit.rz(2*phi, qr[j])
                            circuit.cx(qr[k], qr[j])  


        # RZ's
        # first nr rz
        for j in range(nr):
            phi = (np.pi-v[j])
            circuit.rz(2*phi,qr[j])
        
        phi_20 = (np.pi-np.sum(v))
        circuit.rz(2*phi_20,qr[nq-1])
                         

        
        # H's
        for j in range(nq):
            circuit.h(qr[j])
               
        
        v = v2
        
        # U
        

        # ZZ's, pairs containing a measurement qubit have phi = pi/2
        for j in range(nq):
            for k in range(j,nq):
                phi = (np.pi/2)
                if (j != k):
                    if (j < nr and k < nr):
                        if (j < k):
                            phi = v[j]*(np.pi-v[k])
                        else:
                            phi = (np.pi-v[j]+v[k])
                    elif (j < nr and k >= nr): #try something with meas qubits
                        phi = v[j]*(np.pi-v[k-nr]+v[j])

                    # AVOID SWAPS
                    if ((j==0 and k==1) or (j==0 and k==14) or (j==1 and k==2) or (j==1 and k==13) or (j==2 and k==3) or
                        (j==2 and k==12) or (j==3 and k==4) or (j==3 and k==11) or (j==4 and k==5) or (j==4 and k==10) or
                        (j==5 and k==6) or (j==5 and k==9) or (j==6 and k==8) or (j==7 and k==8)):
                        
                        # ZZ gate
                        if (k != nq-1):
                            circuit.cx(qr[j], qr[k])
                            circuit.rz(2*phi, qr[k])
                            circuit.cx(qr[j], qr[k])
                        else:                           
                            circuit.cx(qr[k], qr[j])
                            circuit.rz(2*phi, qr[j])
                            circuit.cx(qr[k], qr[j])  


        # RZ's
        # first nr rz
        for j in range(nr):
            phi = (np.pi-v[j])
            circuit.rz(2*phi,qr[j])
        
        phi_20 = (np.pi-np.sum(v))
        circuit.rz(2*phi_20,qr[nq-1])
            
            
        # Measure
        circuit.measure(qr[nq-1],cr[m])
        m+=1
        
        i+=2
                 
    return circuit

Transpile 

In [46]:
st = time.time()


# provider = IBMQ.get_provider(hub='ibm-q-research', group='princeton-uni-4', project='main')

# provider = IBMQ.get_provider(hub='ibm-q-research', group='princeton-uni-1', project='main')

# provider = IBMQ.get_provider(hub='ibm-q-community', group='ug-projects', project='princeton-tureci')


provider = IBMQ.get_provider(hub='ibm-q-research', group='princeton-uni-1', project='mid-circuit-meas')

# qcomp = provider.get_backend('ibmq_casablanca')  

qcomp = provider.get_backend('ibmq_16_melbourne')  


circs = []

# Max 900 circuits on Casablanca so let's do first 200 circs on 100 train 0's and 100 train 1's

train_data = np.load('MNISTcwtrain1000.npy')
train_data = train_data.astype(dtype='float64')
test_data = np.load('MNISTcwtest100.npy')
test_data = test_data.astype(dtype='float64')

# resize to desired size
o_size = 28
new_size = 28

new_train_data = resize_dataset(train_data, o_size, new_size)
new_test_data = resize_dataset(test_data, o_size, new_size)

# restrict to 0 and 1
train = new_train_data
test = new_test_data

# normalize grayscale
train = train/255.0
test = test/255.0


nr = 14
nc = int(new_size/nr)*new_size

start=2000
finish=3500
img_ns = np.arange(start,finish)
# img_ns = np.arange(0,450)


for img_n in img_ns:

#     img = unpackcw(test[:,img_n],new_size,new_size)

    img = unpackcw(train[:,img_n],new_size,new_size)

    new_img = reshape_img_MNIST(img,new_size,nr,nc)

    circs.append(circuit_build(new_img))
        
# Need to transpile the circuits first.
circs = transpile(circs, backend=qcomp)



end = time.time()
print(end - st)


1940.9562668800354


In [47]:
job_name="QRC_MNIST_{}x{}_nq15_14x28_20ZZ_Train_imgs_{}_{}_Casablanca".format(new_size,new_size,start,finish)
# job_name="QRC_MNIST_{}x{}_nq15_14x28_20ZZ_Test_imgs_{}_{}_Casablanca".format(new_size,new_size,start,finish)

In [48]:
# Use Job Manager to break the circuits into multiple jobs.
job_manager = IBMQJobManager()
job_set = job_manager.run(circs, backend=qcomp, name=job_name)

In [49]:
print(job_set.report())

Job set name: QRC_MNIST_28x28_nq15_14x28_20ZZ_Train_imgs_2000_3500_Casablanca
          ID: 5c3fb5c8f9b54e11b4f0781c200ad600-1619323129811892
        tags: []
Summary report:
       Total jobs: 20
  Successful jobs: 0
      Failed jobs: 0
   Cancelled jobs: 0
     Running jobs: 0
     Pending jobs: 20

Detail report:
  experiments: 0-74
    job index: 0
    job ID: 6084e91c262a53e1dca07bf4
    name: QRC_MNIST_28x28_nq15_14x28_20ZZ_Train_imgs_2000_3500_Casablanca_0_
    status: job is queued
    queue position: None
  experiments: 75-149
    job index: 1
    job ID: 6084e94d07a9f81880b49f19
    name: QRC_MNIST_28x28_nq15_14x28_20ZZ_Train_imgs_2000_3500_Casablanca_1_
    status: job is queued
    queue position: None
  experiments: 150-224
    job index: 2
    job ID: 6084e98f262a531988a07bf8
    name: QRC_MNIST_28x28_nq15_14x28_20ZZ_Train_imgs_2000_3500_Casablanca_2_
    status: job is queued
    queue position: None
  experiments: 225-299
    job index: 3
    job ID: 6084ed976a3af02be4

managedjob._async_submit:WARNING:2021-04-25 00:50:53,818: Job limit reached, waiting for job 6084c08163ee99d137f09958 to finish before submitting the next one.


In [76]:
jTrain_0_500 = qcomp.retrieve_job("6083a605b31191f9286086ca")
jTrain_500_1000 = qcomp.retrieve_job("6083a6f93cf454e2b8c5c830")
jTrain_1000_1500 = qcomp.retrieve_job("6083abea9518d70a6e6e40f9")
jTrain_1500_1800 = qcomp.retrieve_job("6083ac3a48457204dbef13e1")
jTrain_1800_2000 = qcomp.retrieve_job("60839fb29518d722616e407f")

j_trains = [jTrain_0_500,jTrain_500_1000,jTrain_1000_1500,jTrain_1500_1800,jTrain_1800_2000]
j_trains_ncrcs_p_lab = [500,500,500,300,200]

jTest_0_200 = qcomp.retrieve_job("60839ff607a9f82877b4945f")

In [79]:
def get_Zs_vecs(n_meas,counts):
    # have to build each count individually
    Zs = []
    for i in range(n_meas):
        Zs.append([])
        
    for key in counts:
        
        counts_key = counts[key]
        for r in range(n_meas):
#             print(key[r])           
            list_vals = int(key[r])*np.ones(counts_key)
            
            Zs[n_meas-r-1].extend(list_vals.tolist())
    return Zs   

def get_cov_upper_tri(vec_Zs,n_meas):
    covm = np.cov(vec_Zs)
    upper_tri = list(covm[np.triu_indices(n_meas)])
    return upper_tri
    
def ret_covs(jobs,ncircs_pj,n_meas,isTrain=True):
    
    if (isTrain == False):
        div = 100
    else:
        div = 1000
        
#     n_meas = 64
    
    label = []
    rc_nodes = []
    
    c=0
    
    for job in range(len(jobs)):
        j = jobs[job]
        ncircs = ncircs_pj[job]
            
        res = j.result()
            
        for i in range(ncircs):
            counts = res.get_counts(int(i))

            Zs = get_Zs_vecs(n_meas,counts)
                
            upp_tricov = get_cov_upper_tri(Zs,n_meas)
                
            upp_tricov.extend(np.mean(Zs,axis=1))
                
#             rc_nodes.append(upp_tricov)

            rc_nodes.append(np.mean(Zs,axis=1))


            label.append(int(c/div))
            
            c+=1
            
    return [rc_nodes,label]
    

In [81]:
n_meas = 16
jobs = j_trains
ncircs_pj = j_trains_ncrcs_p_lab

train = ret_covs(jobs,ncircs_pj,n_meas,isTrain=True)


In [82]:
name = "QRC_Zs_MNIST_8x8_Train2000_nq6_4x8_Casablanca"
save_obj(train, name)

In [83]:
n_meas = 16
jobs = [jTest_0_200]
ncircs_pj = [200]

train = ret_covs(jobs,ncircs_pj,n_meas,isTrain=False)


In [84]:
name = "QRC_Zs_MNIST_8x8_Test200_nq6_4x8_Casablanca"
save_obj(train, name)

In [4]:
for j in range(5):
    for k in range(j,5):
        print("({},{})".format(j,k))

(0,0)
(0,1)
(0,2)
(0,3)
(0,4)
(1,1)
(1,2)
(1,3)
(1,4)
(2,2)
(2,3)
(2,4)
(3,3)
(3,4)
(4,4)
